In [10]:
import lilac as ll

items = [{'question': 'A'}, {'question': 'B'}, {'question': 'C'}]
dataset = ll.from_dicts('local', 'questions', items)


def add_prefix(item):
  return 'Q: ' + item['question']


dataset.map(add_prefix, output_column='question_prefixed')
dataset.to_pandas()


Reading from source dict...: 100%|██████████| 3/3 [00:00<00:00, 2235.37it/s]


Dataset "questions" written to ./data/datasets/local/questions
Scheduling task "f67b435a3e01485e8489f8554357e3d7": "[local/questions][1 shards] map "add_prefix" to "question_prefixed"".
Wrote map output to ./data/datasets/local/questions/question_prefixed-00000-of-00001.parquet


[Shard 0/1] map "add_prefix" to "('question_prefixed',)": 6it [00:00, 478.71it/s]              


,question,question_prefixed
0,A,Q: A
1,B,Q: B
2,C,Q: C


Task finished "f67b435a3e01485e8489f8554357e3d7": "[local/questions][1 shards] map "add_prefix" to "question_prefixed"" in 5s.


In [11]:
def add_prefix(question):
  return 'Q: ' + question


dataset.map(add_prefix, input_path='question', output_column='question_prefixed2')
dataset.to_pandas()


Scheduling task "48f71686ebbf4fb8a840fa780bc1f6ea": "[local/questions][1 shards] map "add_prefix" to "question_prefixed2"".
Wrote map output to ./data/datasets/local/questions/question_prefixed2-00000-of-00001.parquet


[Shard 0/1] map "add_prefix" to "('question_prefixed2',)": 100%|██████████| 3/3 [00:00<00:00, 461.89it/s]


,question,question_prefixed,question_prefixed2
0,A,Q: A,Q: A
1,B,Q: B,Q: B
2,C,Q: C,Q: C


Task finished "48f71686ebbf4fb8a840fa780bc1f6ea": "[local/questions][1 shards] map "add_prefix" to "question_prefixed2"" in 5s.


In [3]:
import lilac as ll

items = [
  {'questions': ['A', 'B']},
  {'questions': ['C']},
  {'questions': ['D', 'E']},
]
dataset = ll.from_dicts('local', 'tutorial', items, overwrite=True)
dataset.to_pandas()


Reading from source dict...: 100%|██████████| 3/3 [00:00<00:00, 1989.39it/s]

Dataset "tutorial" written to ./data/datasets/local/tutorial


,questions
0,"[A, B]"
1,[C]
2,"[D, E]"


In [4]:
import lilac as ll

ll.set_project_dir('./data')


def add_prefix(question):
  return 'Q: ' + question


dataset = ll.get_dataset('local', 'tutorial')
dataset.map(
  add_prefix, input_path=('questions', '*'), output_column='questions_with_prefix', overwrite=True
)
dataset.to_pandas()


Scheduling task "9133bf63e9e84c5aa3f1a273d7981659": "[local/tutorial][1 shards] map "add_prefix" to "questions_with_prefix"".


/Users/dsmilkov/code/lilac/.venv/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 58022 instead
  warnings.warn(


Wrote map output to ./data/datasets/local/tutorial/questions_with_prefix-00000-of-00001.parquet


[Shard 0/1] map "add_prefix" to "('questions_with_prefix',)": 100%|██████████| 3/3 [00:00<00:00, 638.82it/s]


,questions,questions_with_prefix
0,"[A, B]","[Q: A, Q: B]"
1,[C],[Q: C]
2,"[D, E]","[Q: D, Q: E]"


Task finished "9133bf63e9e84c5aa3f1a273d7981659": "[local/tutorial][1 shards] map "add_prefix" to "questions_with_prefix"" in 7s.


In [10]:
import lilac as ll

ll.set_project_dir('./data')

items = [
  {
    'text': 'Apple Inc. is an American multinational technology company headquartered in Cupertino, California, (Company: Apple Inc.) that designs, develops, and sells consumer electronics, computer software, and online services.'
  },
  {
    'text': 'Google LLC is an American multinational technology company (Company: Google LLC) that specializes in Internet-related services and products, which include online advertising technologies, a search engine, cloud computing, software, and hardware.'
  },
]
config = ll.DatasetConfig(
  namespace='local',
  name='tutorial3',
  source=ll.DictSource(items),
)
dataset = ll.create_dataset(config, overwrite=True)
dataset.to_pandas()


def extract_company_with_coordinates(text):
  import re

  pattern = r'\(Company: ([^\)]+)\)'
  matches = re.finditer(pattern, text)
  return [ll.span(m.start(), m.end(), {'Company': m.group(1)}) for m in matches]


dataset.map(
  extract_company_with_coordinates, input_path='text', output_column='company', overwrite=True
)


Reading from source dict...: 100%|██████████| 2/2 [00:00<00:00, 1251.66it/s]

Dataset "tutorial3" written to ./datasets/local/tutorial3
Scheduling task "22c95a8997ef490d8a28e56f9933cf94": "[local/tutorial3][1 shards] map "extract_company_with_coordinates" to "company"".
Wrote map output to ./datasets/local/tutorial3/company-00000-of-00001.parquet


[Shard 0/1] map "extract_company_with_coordinates" to "('company',)": 100%|██████████| 2/2 [00:00<00:00, 849.65it/s]


In [21]:
df = dataset.to_pandas()
print(df.columns)


Index(['text', 'company.*', 'company.*.Company'], dtype='object')


In [18]:
df[:1]['company.*.Company']


0    [Apple Inc.]
Name: company.*.Company, dtype: object

In [ ]:
ll.start_server()


INFO:     Started server process [65094]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5432 (Press CTRL+C to quit)


In [ ]:
ll.start_server()


INFO:     Started server process [65547]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5432 (Press CTRL+C to quit)


In [2]:
import lilac as ll

config = ll.DatasetConfig(
  namespace='local',
  name='companies',
  source=ll.HuggingFaceSource(dataset_name='atmallen/companies_azaria_mitchell'),
)
dataset = ll.create_dataset(config, overwrite=True)
dataset.to_pandas()


/Users/dsmilkov/code/lilac/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset "companies" written to ./data/datasets/local/companies


,statement,label,__hfsplit__,__rowid__,overview.sbert.*,overview.openai.*,overview.sbert.*.embedding,overview.openai.*.embedding
0,Shell operates in the industry of oil & gas op...,true,train,a151a69e-3efe-4fa8-a0a9-302f8c9a9fe0,None,None,None,None
1,Nestle operates in the industry of Banking.,false,train,cd9f8dd0-3ff0-4d45-a8ff-39bcd6a87e9d,None,None,None,None
2,Itau Unibanco Holding operates in the industry...,true,train,602ceff5-1973-44fd-911c-fc83a1096948,None,None,None,None
3,Raytheon Technologies operates in the industry...,true,train,7990194a-6e50-45b5-97e6-bc2793b95ede,None,None,None,None
4,Manulife operates in the industry of insurance.,true,train,8de5f7d2-9d63-42cd-bb64-19f911357166,None,None,None,None
...,...,...,...,...,...,...,...,...
1195,Alphabet has headquarters in South Korea.,false,test,aee1baa1-79d5-49cc-8a7a-45199421872a,None,None,None,None
1196,"Iberdrola is a holding company, which engages ...",false,test,0e31deea-2000-46ce-9579-50991f953257,None,None,None,None
1197,Dow has headquarters in Italy.,false,test,b72e5655-7933-4282-b743-4e405defc58f,None,None,None,None
1198,Merck & Co. engages in the provision of health...,true,test,6b39b493-1857-47df-9485-55b21cb9bd1c,None,None,None,None


In [ ]:
ll.start_server()


INFO:     Started server process [69441]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5432 (Press CTRL+C to quit)
[Shard 0/1] map "extract_company" to "('company',)":  53%|█████▎    | 636/1200 [18:29<06:22,  1.47it/s]/var/folders/zn/brkm752x5673zb89tjjqzzs00000gn/T/ipykernel_69441/3762146513.py:33: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
[Shard 0/1] map "extract_company" to "('company',)":  67%|██████▋   | 803/1200 [20:44<06:13,  1.06it/s]/it]

In [1]:
import instructor
from openai import OpenAI
from pydantic import BaseModel, Field
import lilac as ll

dataset = ll.get_dataset('local', 'companies')


def extract_company(text):
  class CompanyDetail(BaseModel):
    """Details of a company."""

    name: str = Field(description='Company name')
    age: int = Field(description='The estimated age of the company')

  client = instructor.patch(OpenAI())
  company = client.chat.completions.create(
    model='gpt-3.5-turbo',
    response_model=CompanyDetail,
    messages=[
      {'role': 'user', 'content': text},
    ],
  )
  return company.model_dump()


dataset.map(extract_company, input_path='statement', output_column='company')


/Users/dsmilkov/code/lilac/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Cannot map to path "company" which already exists in the dataset. Use overwrite=True to overwrite the column.

In [12]:
items = [
  {'questions': ['A', 'B']},
  {'questions': ['C']},
  {'questions': ['D', 'E']},
]
dataset = ll.from_dicts('local', 'nested_questions', items)
dataset.to_pandas()


Reading from source dict...: 100%|██████████| 3/3 [00:00<00:00, 1007.36it/s]

Dataset "nested_questions" written to ./data/datasets/local/nested_questions


,questions
0,"[A, B]"
1,[C]
2,"[D, E]"


In [13]:
def add_prefix(question):
  return 'Q: ' + question


dataset.map(add_prefix, input_path=('questions', '*'), output_column='questions_prefixed')
dataset.to_pandas()


Scheduling task "793083ae34d441adb74aaf6be5602066": "[local/nested_questions][1 shards] map "add_prefix" to "questions_prefixed"".
Wrote map output to ./data/datasets/local/nested_questions/questions_prefixed-00000-of-00001.parquet


[Shard 0/1] map "add_prefix" to "('questions_prefixed',)": 100%|██████████| 3/3 [00:00<00:00, 1097.41it/s]


,questions,questions_prefixed
0,"[A, B]","[Q: A, Q: B]"
1,[C],[Q: C]
2,"[D, E]","[Q: D, Q: E]"


Task finished "793083ae34d441adb74aaf6be5602066": "[local/nested_questions][1 shards] map "add_prefix" to "questions_prefixed"" in 5s.


In [14]:
items = [
  {'question': 'How are you today?'},
  {'question': 'What kind of food'},
  {'question': 'Are you sure?'},
]
dataset = ll.from_dicts('local', 'questions3', items)


def enrich(question):
  return {'length': len(question), 'ends_with_?': question[-1] == '?'}


dataset.map(enrich, input_path='question', output_column='metadata')
dataset.to_pandas()


Reading from source dict...: 100%|██████████| 3/3 [00:00<00:00, 1872.46it/s]


Dataset "questions3" written to ./data/datasets/local/questions3
Scheduling task "2b61d10029d24c74a47e040106e9af0e": "[local/questions3][1 shards] map "enrich" to "metadata"".
Wrote map output to ./data/datasets/local/questions3/metadata-00000-of-00001.parquet


[Shard 0/1] map "enrich" to "('metadata',)": 100%|██████████| 3/3 [00:00<00:00, 370.97it/s]


,question,metadata.length,metadata.ends_with_?
0,How are you today?,18,True
1,What kind of food,17,False
2,Are you sure?,13,True


Task finished "2b61d10029d24c74a47e040106e9af0e": "[local/questions3][1 shards] map "enrich" to "metadata"" in 5s.


In [15]:
ll.start_server()


INFO:     Started server process [5352]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5432 (Press CTRL+C to quit)


In [19]:
items = [
  {'text': 'Company name: Apple Inc.\n Apple Inc is a ...'},
  {'text': 'Google LLC is a ... Company name: Google LLC'},
  {'text': 'There is no company name here'},
]
dataset = ll.from_dicts('local', 'company', items, overwrite=True)


def extract_company(text):
  import re

  pattern = r'Company name: (.*)?\s'
  matches = re.finditer(pattern, text)
  return [ll.span(m.span(1)[0], m.span(1)[1]) for m in matches]


dataset.map(extract_company, input_path='text', output_column='company', overwrite=True)
dataset.to_pandas()


Reading from source dict...: 100%|██████████| 3/3 [00:00<00:00, 2278.27it/s]


Dataset "company" written to ./data/datasets/local/company
Scheduling task "a61e0023e8bb473298ba9e47c6c284f1": "[local/company][1 shards] map "extract_company" to "company"".
Wrote map output to ./data/datasets/local/company/company-00000-of-00001.parquet


[Shard 0/1] map "extract_company" to "('company',)": 100%|██████████| 3/3 [00:00<00:00, 666.19it/s]


,text,company
0,Company name: Apple Inc.\n Apple Inc is a ...,"[{'__span__': {'start': 14, 'end': 24}}]"
1,Google LLC is a ... Company name: Google LLC,"[{'__span__': {'start': 34, 'end': 40}}]"
2,There is no company name here,[]


Task finished "a61e0023e8bb473298ba9e47c6c284f1": "[local/company][1 shards] map "extract_company" to "company"" in 0s.
